In [ ]:
import numpy as np
import xarray as xr
from pathlib import Path
from netCDF4 import Dataset
import argparse
import matplotlib.pyplot as plt
import matplotlib.tri as tri
import matplotlib.gridspec as gridspec
from matplotlib.colorbar import Colorbar
from matplotlib.colors import Normalize, TwoSlopeNorm
import glob
import os

import cartopy.crs as ccrs
import cartopy
#import cartopy.mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rcParams, cycler
from matplotlib import animation, rc
from matplotlib.gridspec import GridSpec
from mpl_toolkits.axes_grid1.inset_locator import inset_axes


from xeofs.xarray import EOF


import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rcParams, cycler
from matplotlib import animation, rc
from matplotlib.gridspec import GridSpec
from mpl_toolkits.axes_grid1.inset_locator import inset_axes


import scipy
from scipy import signal
from shapely.geometry import mapping
from xarrayutils.utils import linear_trend, xr_linregress
import pandas as pd
import geopandas as gpd
import cmocean


In [ ]:

main_dir = Path.cwd().parent # Main directory path of project repository - all filepaths are relative to this

# File path directories
DIR_external = 'data/external/'

# DATASET FILEPATHS
# Basal melt observations from Paolo 2023
DIR_basalMeltObs = 'data/external/Paolo2023/'
# Ocean model output - E3SM (SORRMv2.1.ISMF), data received from Darin Comeau / Matt Hoffman at LANL
DIR_SORRMv21 = 'data/external/SORRMv2.1.ISMF/regridded_output/'

# DATA FILENAMES
FILE_MeltDraftObs = 'ANT_G1920V01_IceShelfMeltDraft.nc'
FILE_SORRMv21 = 'Regridded_SORRMv2.1.ISMF.FULL.nc'
FILE_SORRMv21_DETRENDED = 'SORRMv21_detrended.nc'
FILE_iceShelvesShape = 'iceShelves.geojson'

# INTERIM GENERATED FILEPATHS
DIR_basalMeltObs_Interim = 'data/interim/Paolo2023/iceShelves_dedraft/iceShelfRegions/'
DIR_SORRMv21_Interim = 'data/interim/SORRMv2.1.ISMF/iceShelves_dedraft/iceShelfRegions/'


In [ ]:
obs23_clean = xr.open_dataset(main_dir / DIR_basalMeltObs_Interim / "obs23_clean.nc")
sorrmv21_clean = xr.open_dataset(main_dir / DIR_SORRMv21_Interim / "sorrmv21_clean.nc")

In [ ]:
sorrmv21_clean = sorrmv21_clean.rename({"__xarray_dataarray_variable__":"flux", "Time":"time"})

In [ ]:
crs = ccrs.SouthPolarStereo();

In [ ]:
flux_clean = sorrmv21_clean.flux[3000:9000]

In [ ]:
sorrm_ts = xr.open_dataset(main_dir / DIR_SORRMv21_Interim / "SORRMv21_CLEAN_TS.nc")

In [ ]:
sorrm_ts = sorrm_ts.__xarray_dataarray_variable__

In [ ]:
import statsmodels

In [ ]:
from statsmodels.tsa.seasonal import STL

stl = STL(sorrm_ts[:], period=13)
res = stl.fit()
fig = res.plot()

In [ ]:
def add_stl_plot(fig, res, legend):
    """Add 3 plots from a second STL fit"""
    axs = fig.get_axes()
    comps = ["trend", "seasonal", "resid"]
    for ax, comp in zip(axs[1:], comps):
        series = getattr(res, comp)
        if comp == "resid":
            ax.plot(series, marker="o", linestyle="none")
        else:
            ax.plot(series)
            if comp == "trend":
                ax.legend(legend, frameon=False)

stl = STL(sorrm_ts, period=12, robust=True)
res_robust = stl.fit()
fig = res_robust.plot()
# res_non_robust = STL(sorrm_ts, period=12, robust=False).fit()
# add_stl_plot(fig, res_non_robust, ["Robust", "Non-robust"])

In [ ]:
fig = plt.figure(figsize=(16, 5))
lines = plt.plot(res_robust.weights, marker="o", linestyle="none")
ax = plt.gca()

In [ ]:
model = EOF(flux_clean)
model.solve()
eofs = model.eofs()
pcs = model.pcs()
nmodes = model.n_modes
varexpl = model.explained_variance_ratio()

In [ ]:
eofs.to_netcdf(main_dir / "data/interim/SORRMv2.1.ISMF/EOF_PCA_modes/" / "sorrmv21_eofs.nc" )
pcs.to_netcdf(main_dir / "data/interim/SORRMv2.1.ISMF/EOF_PCA_modes/" / "sorrmv21_pcs.nc" )
varexpl.to_netcdf(main_dir / "data/interim/SORRMv2.1.ISMF/EOF_PCA_modes/" / "sorrmv21_varexpl.nc" )

In [ ]:
# Normalize 
flux_clean_tmean = flux_clean.mean('time')

In [ ]:
flux_clean_tstd = flux_clean.std('time')

In [ ]:
flux_clean_demeaned = flux_clean - flux_clean_tmean


In [ ]:
flux_clean_normalized = flux_clean_demeaned/flux_clean_tstd

In [ ]:
flux_clean_normalized.to_netcdf(main_dir / "data/interim/SORRMv2.1.ISMF/" / "flux_clean_6000_normalized.nc")
flux_clean_tstd.to_netcdf(main_dir / "data/interim/SORRMv2.1.ISMF/" / "flux_clean_6000_tstd.nc")

In [ ]:
flux_clean_normalized = xr.open_dataset(main_dir / "data/interim/SORRMv2.1.ISMF/" / "flux_clean_6000_normalized.nc")

In [ ]:
flux_clean_normalized = flux_clean_normalized.flux

In [ ]:
# Normalized variability decomposition 
norm_model = EOF(flux_clean_normalized)
norm_model.solve()
norm_eofs = norm_model.eofs()
norm_pcs = norm_model.pcs()
norm_nmodes = norm_model.n_modes
norm_varexpl = norm_model.explained_variance_ratio()
Fvn_cs = norm_varexpl.cumsum()


In [ ]:
norm_pcs_eig = norm_model.pcs(1)

In [ ]:
norm_eofs.to_netcdf(main_dir / "data/interim/SORRMv2.1.ISMF/EOF_PCA_modes/" / "sorrmv21_norm_eofs.nc" )
norm_pcs.to_netcdf(main_dir / "data/interim/SORRMv2.1.ISMF/EOF_PCA_modes/" / "sorrmv21_norm_pcs.nc" )
norm_varexpl.to_netcdf(main_dir / "data/interim/SORRMv2.1.ISMF/EOF_PCA_modes/" / "sorrmv21_norm_varexpl.nc" )

In [ ]:
# Plot figure
# Figure: EOF modes from decomposition of normalized model variability data

nmodes_plot = 4 # Number of modes to plot

yr_length = range(0,125)
yrs_plot = np.repeat(yr_length, 12)

#sns.set_theme()
sns.set_theme(style="whitegrid")
fig=plt.figure(figsize=(12,8))
gs = GridSpec(nmodes_plot,2,width_ratios=[1.5,2])
axbig = fig.add_subplot(gs[0,:])
ax0 = [fig.add_subplot(gs[i,0],projection=crs) for i in range(1,nmodes_plot)]
ax1 = [fig.add_subplot(gs[i,1]) for i in range(1,nmodes_plot)]

for i, (a0, a1) in enumerate(zip(ax0, ax1)):
    norm_eofs.sel(mode=i+1).plot(ax=a0,
                            cmap='cmo.balance',
                            add_colorbar=True,
                            cbar_kwargs={'orientation': 'vertical',
                                         'label': 'SD Units'})
    a0.coastlines(resolution='10m',linewidth=0.25,color='0.8')
    a0.set_title('EOF Mode {}'.format(i+1))
    #a1.set_ylim(-3,3)
    a1.plot(norm_pcs.sel(mode=i+1),linewidth=1,color='dimgray')
    a1.set_xlabel('')
    a1.set_title('PC Mode {}'.format(i+1))
    a1.tick_params(labelbottom=False, left=False, grid_alpha=0.3)

a1.set_xlabel('Time (months)');

m = 500
xvar = np.linspace(1,m,m)
axbig.plot(xvar,Fvn_cs[:m]*100, 'k*--', lw=0.5,markersize=1.75,
         label='Actual Data: {:.2f}\%'.format(norm_varexpl[:m].sum().values*100))
axbig.set_ylabel('Cumulative variance captured (\%)')
axbig.set_xlabel('Mode Number')
axbig.set_title('Cumulative variance captured by first {} modes'.format(m));
axbig.tick_params(labelbottom=True, left=False, grid_alpha=0.3)
axbig.legend();

fig.tight_layout()

In [ ]:
sorrmv21_clean = xr.open_dataset(main_dir / DIR_SORRMv21_Interim / "sorrmv21_clean.nc", chunks={"Time":300})

In [ ]:
if 'time' in ds.dims:
    tdim = 'time'
elif 'Time' in ds.dims:
    tdim = 'Time'

In [ ]:
obs23_ts = obs23_clean.sum(["x","y"])
# sorrmv21_ts = sorrmv21_clean.sum(["x","y"])

In [ ]:
dsmonth = ds.groupby("Time.month")

In [ ]:
dsmonth

In [ ]:


# Deseasonalize
# Remove climatologies to isolate anomalies / deseasonalize 
flux_month = sorrmv21_clean.groupby("time.month")
flux_clm = flux_month.mean("time") # Climatologies
flux_anm = flux_month - flux_clm # Deseasonalized anomalies

In [ ]:
sorrmv21_ts = sorrmv21_clean.__xarray_dataarray_variable__.sum(["x","y"]).compute()

In [ ]:
plt.figure(figsize=(25,8))
sorrmv21_ts.plot()

In [ ]:
plt.figure(figsize=(15,8))
plt.psd(obs23_ts.melt);
plt.psd(sorrmv21_ts);

In [ ]:
plt.psd(obs23_ts.melt);
plt.psd(sorrm_ts[3000:9000]);

In [ ]:
plt.figure(figsize=(25,8))
plt.plot(sorrm_ts[3000:9000])

In [ ]:
DIR_SORRMv21 = Path("data/interim/SORRMv2.1.ISMF/iceShelves_dedraft/iceShelfRegions/")
main_dir / DIR_SORRMv21.parent / FILE_SORRMv21_DETRENDED

In [ ]:
sorrm_clean_ts = xr.open_dataset(main_dir / DIR_SORRMv21 / "SORRMv21_CLEAN_TS.nc")
sorrm_ts = sorrm_clean_ts.__xarray_dataarray_variable__

In [ ]:
plt.figure(figsize=(12,8))
#ds.__xarray_dataarray_variable__.plot()
plt.psd(ds.__xarray_dataarray_variable__[8000:10000]);
plt.psd(ds.__xarray_dataarray_variable__[3000:9000]);
# plt.psd(obs23_ts.melt);
plt.psd(ds.__xarray_dataarray_variable__);
# plt.psd(sorrmv21_ts);

In [ ]:
# STANDARDIZE

obs23_ts_mean = obs23_ts.mean()
obs23_ts_std = obs23_ts.std()
obs23_ts_norm = (obs23_ts - obs23_ts_mean)/obs23_ts_std

sorrm_ts_mean = sorrm_ts.mean()
sorrm_ts_std = sorrm_ts.std()
sorrm_ts_norm = (sorrm_ts - sorrm_ts_mean)/sorrm_ts_std


In [ ]:
obs23_ts_norm.melt

In [ ]:
plt.psd(obs23_ts_norm.melt);
plt.psd(sorrm_ts_norm[3000:9000]);

In [ ]:
plt.figure(figsize=(25,8))
plt.plot(obs23_ts_norm.melt)
plt.plot(sorrm_ts_norm[:250], '--')

In [ ]:
main_dir / DIR_SORRMv21.parent

In [ ]:
ctrlRegional = xr.open_dataset(ctrl+regionalStats)
exp01Regional = xr.open_dataset(exp01+regionalStats)

#ctrlglobal = xr.open_dataset(ctrl+globalStats)
exp01global = xr.open_dataset(exp01+globalStats)

In [ ]:
ds = xr.open_dataset(dir1)
ds2 = xr.open_dataset(dir2)
ds3 = xr.open_dataset(dir3)
ds.deltat[:50].plot(label='obs23MALI (50 of 325 timesteps)')
ds2.deltat.plot(label='origMALI')
tmpds.deltat.plot(label='obs23MALI (Modified timestep)')
#ds3.deltat[:50].plot(label='obs23-1101-MALI')
plt.title('ismip6/hist04 runs')
plt.legend()

In [ ]:
#ds.surfaceSpeedMax[:50].plot(label='obs23MALI (100 of 325 timesteps)')
ds2.surfaceSpeedMax.plot(label='origMALI')
tmpds.surfaceSpeedMax.plot(label='obs23MALI (Modified timestep)')
#ds3.deltat[:50].plot(label='obs23-1101-MALI')
plt.title('ismip6/hist04 runs')
plt.legend()

In [ ]:
MELTDRAFT_OBS = xr.open_dataset(main_dir / DIR_basalMeltObs / FILE_MeltDraftObs, chunks={'x':729, 'y':729})

In [ ]:
MELTDRAFT_OBS.rio.write_crs("epsg:3031",inplace=True);


In [ ]:
h = MELTDRAFT_OBS.draft
if 'time' in h.dims:
    tdim = 'time'
elif 'Time' in h.dims:
    tdim = 'Time'

h_mean = h.mean(tdim)

IMBIEregions = range(6,33)
iceShelfRegions = range(33,133)

In [ ]:
mlt = MELTDRAFT_OBS.melt.rio.clip(icems.loc[[34],'geometry'].apply(mapping),icems.crs,drop=False)
mlt_mean = mlt.mean(tdim)
# Dedraft: Linear Regression with SSH over chosen basin
print('calculating linear regression for catchment {}'.format(icems.name.values[i]))
mlt_rgrs = xr_linregress(h, mlt_mean, dim=tdim) # h = independent variable

In [ ]:
mlt_prd = mlt_rgrs.slope*h_mean + mlt_rgrs.intercept

In [ ]:
icems.name.values

In [ ]:
DIR_external = 'data/external/'
DIR_interim = 'data/interim/'

# DATASET FILEPATHS
# Ocean model output - E3SM (SORRMv2.1.ISMF), data received from Darin Comeau / Matt Hoffman at LANL
DIR_SORRMv21 = 'data/external/SORRMv2.1.ISMF/regridded_output/'
FILE_SORRMv21 = 'Regridded_SORRMv2.1.ISMF.FULL.nc'
FILE_SORRMv21_DETRENDED_TS = 'SORRMv21_detrended_ts.nc'

ds = xr.open_dataset(main_dir / DIR_SORRMv21 / FILE_SORRMv21_DETRENDED_TS)


In [ ]:
ds["flux_sum"] = ds.__xarray_dataarray_variable__

In [ ]:
plt.figure(figsize=(25,8))
ds.__xarray_dataarray_variable__.plot()

In [ ]:
ds.__xarray_dataarray_variable__.rename("flux_xy_sum")

In [ ]:
ds